# Recurrent Units Without Matrix Multiplications

```{article-info}
:avatar: https://avatars.githubusercontent.com/u/25820201?v=4
:avatar-link: https://github.com/PhotonicGluon/
:author: "[Ryan Kan](https://github.com/PhotonicGluon/)"
:date: "Jul 4, 2024"
:read-time: "{sub-ref}`wordcount-minutes` min read"
```

This page explains the theory behind `GRUMML` and `LRUMML`.

## A Quick Primer on the Gated Recurrent Unit (GRU)

The Long Short-Term Memory (LSTM) architecture is a staple layer in recurrent neural networks (RNNs), dating back to 1995 in the landmark paper *Long Short Term Memory* by Hochreiter and Schmidhuber. The [main LSTM paper](https://www.researchgate.net/publication/13853244) was subsequently published in 1997 in the journal *Neural Computation*, and since then minor modifications to the original LSTM implementation have been made.

The GRU was a comparatively recent addition to the RNN family by Cho et al. in the paper [*Learning Phrase Representations using RNN Encoder–Decoder
for Statistical Machine Translation*](https://arxiv.org/pdf/1406.1078v1). An empirical evaluation of GRUs against LSTMs was performed in the follow-up paper [*Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling*](https://arxiv.org/pdf/1412.3555v1) by Chung et al. showed that GRUs are comparable to LSTMs on their evaluation tasks. The appeal of GRUs over LSTMs is its (relative) simplicity, using less computation and operations to achieve similar results.

We can formalize the GRU as follows[^gru-notation]. Let $n$ be the input dimension and $d$ be the model dimension (i.e., state dimension). Suppose we have a sequence of $T$ vectors, where $\mathbf{x}_t \in \mathbb{R}^n$ is the input vector at time step $t$ (where $1 \leq t \leq T$). The recurrence is thus
$$
\begin{align*}
    \mathbf{r}_t &= \sigma(\mathbf{x}_t\mathbf{W}_{xr} + \mathbf{h}_{t-1}\mathbf{W}_{hr} + \mathbf{b}_r)\\
    \mathbf{f}_t &= \sigma(\mathbf{x}_t\mathbf{W}_{xf} + \mathbf{h}_{t-1}\mathbf{W}_{hf} + \mathbf{b}_f)\\
    \mathbf{c}_t &= \tau(\mathbf{x}_t\mathbf{W}_{xc} + (\mathbf{r}_t \odot \mathbf{h}_{t-1})\mathbf{W}_{cc} + \mathbf{b}_c)\\
    \mathbf{h}_t &= \mathbf{f}_t\odot\mathbf{h}_{t-1} + (1-\mathbf{f}_t)\odot\mathbf{c}_t\\
    \mathbf{o}_t &= \mathbf{h}_t
\end{align*}
$$
where
- $\mathbf{h}_t \in \mathbb{R}^d$ is the hidden state vector at the timestep $t$ (where we define $\mathbf{h}_0 = \mathbf{0}$);
- $\mathbf{r}_t \in \mathbb{R}^d$ can be called the *reset gate vector*;
- $\mathbf{f}_t \in \mathbb{R}^d$ can be called the *forget gate vector*;
- $\mathbf{c}_t \in \mathbb{R}^d$ can be called the *candidate hidden state*;
- $\mathbf{o}_t \in \mathbb{R}^d$ is the output vector;
- $\mathbf{W}_{xr}$, $\mathbf{W}_{xf}$, and $\mathbf{W}_{xc}$ are $n \times d$ weight matrices relating to the input vector;
- $\mathbf{W}_{hr}$, $\mathbf{W}_{hf}$, and $\mathbf{W}_{vc}$ are $d \times d$ weight matrices relating to the hidden state;
- $\mathbf{b}_r$, $\mathbf{b}_f$, and $\mathbf{b}_c$ are bias vectors in $\mathbb{R}^d$;
- $\sigma$ is the sigmoid activation function;
- $\tau$ is a non-linear activation function (e.g., $\tanh$, $\mathrm{SiLU}$); and
- $\odot$ is the Hadamard product (i.e., element-wise product).

[^gru-notation]: The notation used to describe GRUs differs from the aforementioned paper by Cho et al. (and Chung et al. for that matter). We follow the notation used in [*Scalable MatMul-free Language Modeling*](https://arxiv.org/pdf/2406.02528v1) by Zhu et al. as that would make it easier to describe the changes made to implement a matmul-less version later.

One can see from the equations above that there are a lot of matrix multiplications involved in a GRU. However, note that the recurrence computing $\mathbf{h}_t$ does not involve any matrix multiplications at all, just element-wise multiplication. For a full matmul-less implementation of a GRU, we would like to keep this property while making further changes to truly remove matrix multiplications from the GRU.

## Implementing `GRUMML`

### Standard Implementation

[*Scalable MatMul-free Language Modeling*](https://arxiv.org/pdf/2406.02528v1) by Zhu et al. proposes a few modifications to the GRU implementation in order to reduce (and then remove) matrix multiplications in the GRU layer.

1. Hidden-state related weights $\mathbf{W}_{hr}$, $\mathbf{W}_{hf}$, and $\mathbf{W}_{cc}$ are removed. This is to allow for parallelization similar to that of the Transformer architecture.
2. A data-dependent output gate $\mathbf{g}_t$ is added between the hidden state computation $\mathbf{h}_t$ and the output vector computation $\mathbf{o}_t$. Specifically, the computation for $\mathbf{o}_t$ is modified to become 
    $$
    \begin{align*}
        \mathbf{g}_t &= \sigma(\mathbf{x}_t\mathbf{W}_{xg} + \mathbf{b}_g)\\
        \mathbf{o}_t' &= \mathbf{g}_t \odot \mathbf{h}_t\\
        \mathbf{o}_t &= \mathbf{o}_t'\mathbf{W}_o + \mathbf{b}_o\\
    \end{align*}
    $$
    where we introduce
    - a new weight matrix $\mathbf{W}_{xg}$ of size $n \times d$;
    - a new weight matrix $\mathbf{W}_o$ of size $d \times d$; and
    - new bias vectors $\mathbf{b}_g, \mathbf{b}_o \in \mathbf{R}^d$.
3. All weight matrices are changed to become ternary weight matrices (i.e., the values in the matrices must be in the set $\{-1, 0, 1\}$), allowing us to use ternary multiplication and remove any matrix multiplication.

The resulting GRU architecture (which the authors call $\mathrm{MLGRU}$ but we call `GRUMML`) can be formalized as
$$
\begin{align*}
    \mathbf{f}_t &= \sigma(\mathbf{x}_t\mathbf{W}_f + \mathbf{b}_f)\\
    \mathbf{c}_t &= \tau(\mathbf{x}_t\mathbf{W}_c + \mathbf{b}_c)\\
    \mathbf{g}_t &= \sigma(\mathbf{x}_t\mathbf{W}_g + \mathbf{b}_g)\\
    \mathbf{h}_t &= \mathbf{f}_t\odot\mathbf{h}_{t-1} + (1-\mathbf{f}_t)\odot\mathbf{c}_t\\
    \mathbf{o}_t' &= \mathbf{g}_t \odot \mathbf{h}_t\\
    \mathbf{o}_t &= \mathbf{o}_t'\mathbf{W}_o + \mathbf{b}_o\\
\end{align*}
$$
where
- $\mathbf{W}_f$, $\mathbf{W}_c$, and $\mathbf{W}_g$ are $n \times d$ ternary weight matrices;
- $\mathbf{W}_o$ is a $d \times d$ ternary weight matrix; and
- $\mathbf{b}_f, \mathbf{b}_c, \mathbf{b}_g, \mathbf{b}_o \in \mathbf{R}^d$ are bias vectors.

### Multi-headed `GRUMML`

The main purpose for Zhu et al. to introduce a matmul-less GRU variant is to use it as a multi-headed attention replacement; instead of using multiple matrices to store information about embeddings, the $\mathrm{MLGRU}$ is a variant that uses recurrence to have a similar effect.

Let $H$ denote the number of heads to use. We will split the features contained in the forget gate $\mathbf{f}_t$ and candidate hidden state $\mathbf{c}_t$ and give it to each of the heads; in particular each head now processes $\frac dH$ features (where we assume that $d$ is a multiple of $H$). The hidden states are also split among the multiple heads. This allows for further parallelization.

## Linear Recurrent Units (LRUs)

A key issue with LSTMs and GRUs is that they are slow to optimize. This is due to their sequential nature &mdash; they rely on previous timesteps to generate the current step's output, and so the gradient propagation is slower.

In [*Resurrecting Recurrent Neural Networks for Long Sequences*](https://arxiv.org/pdf/2303.06349v1) by Orvieto et al., an alternative to the GRU was introduced, called the LRU...

TODO: ADD

## Implementing `LRUMML`

TODO: ADD

## OLD - A Quick Primer on Gated Linear Recurrent Layers

The notation for describing recurrent neural networks varies widely from author to author. For this page, we will use a mix of notation from [*Hierarchically Gated Recurrent Neural Network for
Sequence Modeling*](https://arxiv.org/pdf/2404.07904v1) by Qin, Yang, and Zhong and from [*HGRN2: Gated Linear RNNs with State Expansion*](https://arxiv.org/pdf/2404.07904v1) by Qin et al.

Suppose we have an input at timestep $t$ (where $1 \leq t \leq T$ and $T$ denotes the sequence length), which is denoted by $\mathbf{x}_t$, where $\mathbf{x}_t \in \mathbb{R}^n$ (here $n$ denotes the input dimension). A minimal gated linear recurrent layer considers $\mathbf{x}_t$ and the previous *hidden state* $\mathbf{h}_{t-1} \in \mathbb{R}^d$ (here $d$ denotes the model dimension) to produce the next hidden state $\mathbf{h}_t$ and the output $\mathbf{y}_t \in \mathbb{R}^d$. This is done using the recurrence
$$
\begin{align*}
    \mathbf{f}_t &= \sigma\left(\mathbf{x}_t\mathbf{W}_f + \mathbf{b}_f\right)\\
    \mathbf{i}_t &= \sigma\left(\mathbf{x}_t\mathbf{W}_i + \mathbf{b}_i\right)\\
    \mathbf{c}_t &= \tau\left(\mathbf{x}_t\mathbf{W}_c + \mathbf{b}_c\right)\\
    \mathbf{h}_t &= \mathbf{f}_t\odot\mathbf{h}_{t-1} + (1-\mathbf{f}_t)\odot\mathbf{i}_t\\
    \mathbf{y}_t &= \mathbf{h}_t\odot\mathbf{c}_t
\end{align*}
$$
where
- $\mathbf{f}_t$ can be called the *forget gate*;
- $\mathbf{i}_t$ can be called the *input gate*;
- $\mathbf{c}_t$ can be considered to be the candidate hidden state;
- $\mathbf{W}_f$, $\mathbf{W}_i$, and $\mathbf{W}_c$ are $n \times d$ weight matrices;
- $\mathbf{b}_f$, $\mathbf{b}_i$, and $\mathbf{b}_c$ are $d$-dimensional vectors;
- $\sigma$ is the sigmoid activation function;
- $\tau$ is a non-linear activation function (e.g., $\tanh$, $\mathrm{SiLU}$); and
- $\odot$ is the Hadamard product (i.e., elementwise product).